# Np's and Mesogens, Temperature Ramp 6.0 -> 6.2

In [5]:
#-----Import essential packages.

from __future__ import division
import hoomd
import hoomd.md

In [6]:
#-----Define important variables.

p_max = 1.8;
t_0   = 6.0; 
t_1   = 6.2;
steps_ramp = 5e4;

In [7]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(t_0) + "_P_" + str(p_max) + "_ramp.gsd"

In [8]:
#-----Declare a simulation context.

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Initialize from a .gsd file.

system = hoomd.init.read_gsd(ramp_file, frame = -1);

#-----Define each mesogen in the local frame of reference of each rigid body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Create the mesogens.

rigid.create_bodies();

#-----Define the potential energy.

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Declare interactions.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integrate using an NPT integrator.

temp = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 2.0, tauP = 1.0, P = p_max);
npt.randomize_velocities(seed=42)

#-----Save .log y .gsd data.

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "Nano_Particles" created containing 4 particles
notice(2): Group "rigid_center" created containing 1000 particles
notice(2): Group "NP_Mes" created containing 1004 particles


In [9]:
#-----Run the simulation.

hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 755165 / 805000 | TPS 16.3585 | ETA 00:50:46
Time 00:00:20 | Step 755348 / 805000 | TPS 18.2879 | ETA 00:45:15
Time 00:00:30 | Step 755529 / 805000 | TPS 18.07 | ETA 00:45:37
Time 00:00:40 | Step 755708 / 805000 | TPS 17.8678 | ETA 00:45:58
Time 00:00:50 | Step 755895 / 805000 | TPS 18.6617 | ETA 00:43:51
Time 00:01:00 | Step 756083 / 805000 | TPS 18.7933 | ETA 00:43:22
Time 00:01:10 | Step 756267 / 805000 | TPS 18.2887 | ETA 00:44:24
Time 00:01:20 | Step 756450 / 805000 | TPS 18.272 | ETA 00:44:17
Time 00:01:30 | Step 756639 / 805000 | TPS 18.882 | ETA 00:42:41
Time 00:01:40 | Step 756822 / 805000 | TPS 18.2852 | ETA 00:43:54
Time 00:01:50 | Step 757010 / 805000 | TPS 18.7939 | ETA 00:42:33
Time 00:02:00 | Step 757193 / 80500

In [10]:
#-----Get density.

system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x1044c2588>),
             ('particles', <hoomd.data.particle_data at 0x1127e0978>),
             ('number_density', 0.6129374895719425),
             ('bonds', <hoomd.data.bond_data at 0x1127e09b0>),
             ('angles', <hoomd.data.angle_data at 0x1127e0a58>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x1127e0ac8>),
             ('impropers', <hoomd.data.dihedral_data at 0x1127e0b38>),
             ('constraints', <hoomd.data.constraint_data at 0x1127e0ba8>),
             ('pairs', <hoomd.data.bond_data at 0x1127e0c18>),
             ('timestep', 805000)])

In [11]:
#-----Get volume.

system.box.get_volume()

14689.915616498069